In [ ]:
import os
import os.path as osp
import sys
import platform
import shutil
import re
import datetime
import logging
import multiprocessing
from glob import glob
import cv2
import pandas as pd
import trilabtracker as tt

from importlib import reload
# The loop below reloads every submodule. It only works first time if 
# "tt.__all__" has the correct order: if submodule A imports submodule B, 
# then B needs to be reloaded before A.
for m in tt.__all__:
    eval(f'reload(tt.{m})')
reload(tt)

os.getcwd()

### Decide what to track, with what settings, and where to store the output.

In [ ]:
# Videos to track.
input_files  = sorted(glob('../raw_videos/*.m*'))
# input_files   = ['../raw_videos/20190513_SF_n5_t5_3128.mp4']

# Store output in this directory, in a subdirectory named after the video.
tracking_dir = 'tracking_output'

# Load settings.
filtered_settings = tt.load_filtered_settings('settings/adult-schooling.xlsx')

# Load list of trial names to use in the analysis.
valid_trials = open('settings/adult-trials-to-analyze.txt').readlines()
valid_trials = [ fn.strip() for fn in valid_trials ]

# Define how to extract trial info from the video file's path.
# Return None if the trial is to be skipped.
def extract_trial_info(input_file):
    # Extract trial name and number of fish from the video file's path.
    d,fn       = osp.split(input_file)
    fn,ext     = osp.splitext(fn)
    output_dir = osp.join(tracking_dir,fn)
#     n_ind      = int(re.findall('_n(\d+)',fn)[0])
    _,pop,n    = fn.split('_')[:3]
    n_ind      = int(n[1:])
    pop        = pop.lower()
    if 'dark' in fn.lower():
        pop = pop+'-dark'
        # Skip dark trials until I implement one-sided background subtraction.
#         return None
    
    if not fn in valid_trials:
        return None
    
#     # If a repaired version exists, use that instead.
#     repaired_file = osp.join(d,'repaired',fn+'-repaired'+ext)
#     input_file = repaired_file if osp.exists(repaired_file) else input_file

    # If a trial file already exists, skip.
    if osp.exists(osp.join(output_dir,'trial.pik')):
        return None
    
    return dict( input_video=input_file, output_dir=output_dir, 
                 n_ind=n_ind, pop=pop, trial_name=fn )
    
#----------------------------------------------------
# Create settings for input files.

settings_list = []
for input_file in input_files:
    info = extract_trial_info(input_file)
    if info is None:
        continue
    settings = tt.apply_filtered_settings(filtered_settings, info['trial_name'])
    settings.update(info)
    settings_list.append(settings)

# print(f'{len(settings_list)}/{len(input_files)}')
# for settings in settings_list:
#     print(settings['trial_name'])

#----------------------------------------------------
# Count trials of each type.

trials = [ {k:s[k] for k in ['trial_name','n_ind','pop']} for s in settings_list ]
trials = pd.DataFrame(trials) #.set_index('trial_name')
grouped_trials = trials.groupby(['pop','n_ind'])
count  = pd.DataFrame(grouped_trials['trial_name'].count().rename('count'))
count  = count.unstack(1)
count.columns = count.columns.droplevel()
count[pd.isna(count)] = 0
count  = count.astype(int)
display(count)

### Set up output directories. Compute backgrounds.

In [ ]:
# Look for pre-existing background files in this tracking directory.
bkg_dir = None

#----------------------------------------------------

if not osp.exists(tracking_dir):
    os.mkdir(tracking_dir)

for i,settings in enumerate(settings_list):
    tt.overprint(f'{i+1}/{len(settings_list)}  '+settings['trial_name'])
    tracker = tt.Tracker(**settings)
    
    # Create output directory.
    tracker.init_directory()
    tracker.save_settings()
    tracker.init_video_link()
    
    # If background file(s) already exist, don't do anything.
    if osp.exists(str(bkg_dir)):
        for fn in ['background.npz']: #,'background2.npz']:
            f1 = osp.join(bkg_dir, settings['trial_name'], fn)
            f2 = osp.join(tracker.output_dir, fn)
            if osp.exists(f1) and (not osp.exists(f2)) \
               and (fn=='background.npz' or settings['bkg.secondary_subtraction']):
                shutil.copy2(f1,f2)
    
    # Otherwise, compute background.
    if (not osp.exists(osp.join(tracker.output_dir, 'background.npz'))) or \
       (settings['bkg.secondary_subtraction'] and \
          (not osp.exists(osp.join(tracker.output_dir, 'background2.npz')))):
        tracker.init_video_input()
        tracker.init_background()
        tracker.init_secondary_background()
        tracker.release()

### Locate tanks.

In [ ]:
# Look for pre-existing tank files in this tracking directory.
# tank_dir = osp.relpath(osp.join(tracking_dir,'../full_21-01-17')) # None # 
tank_dir = 'tracking_output/archive/21-08-11'

#----------------------------------------------------

for i,settings in enumerate(settings_list):
    tt.overprint(f'{i+1}/{len(settings_list)}  '+settings['trial_name'])
    
    f2 = osp.join(settings['output_dir'], 'tank.pik')
    if osp.exists(f2):
        continue
    
    f1 = osp.join(tank_dir, settings['trial_name'], 'tank.pik')
    if osp.exists(str(tank_dir)):
        if osp.exists(f1) and (not osp.exists(f2)):
            shutil.copy2(f1,f2)
    
    tracker = tt.Tracker(**settings)
    tracker.init_video_input()
    tracker.init_background()
    tracker.release()
    tracker.init_tank(thresh=2,dilate=80)

In [ ]:
# Clean up openCV windows if needed.
cv2.destroyAllWindows()

## Track.

In [ ]:
n_threads = 4
# This multi-threading does not work in Windows. 
# If the OS is Windows, use a single thread.
if n_threads==1 or ('windows' in platform.system().lower()):
    for settings in settings_list:
        tt.reset_logging()
        tt.add_log_stream(sys.stdout, overwrite=True)
        logging.info(settings['input_video']+'\n')
        tt.Tracker(**settings).track_video()
else:
    def track(settings):
        tt.reset_logging()
        return tt.Tracker(**settings).track_video()
    pool = multiprocessing.Pool(n_threads)
    pool.map(track,settings_list)

## View output.

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.patches import Ellipse

# trial_files = glob(osp.join(tracking_dir,'*/trial.pik'))
# print(*trial_files, sep='\n', end='\n\n')
# trial_file = trial_files[0]

# trial = tt.load_trial(trial_file)
# data,frames = trial['data'],trial['frame_list']

# plt.figure(figsize=(6,6))
# xy = trial['data'][:,:,:2]
# for i in range(xy.shape[1]):
#     plt.plot(*xy[:,i,:].T,label=f'{i}')
# xc,yc,R = (trial['tank'][k] for k in ['xc', 'yc', 'R'])
# print(xc,yc,R)
# plt.gca().add_artist(Ellipse((xc,yc), 2*R, 2*R, fc='None', ec='k', lw=0.5))
# plt.legend(loc=(1.05,0.5))
# plt.axis('equal')
# plt.xlim(xc-1.1*R,xc+1.1*R)
# plt.ylim(yc-1.1*R,yc+1.1*R)
# plt.show()